In [1]:
import pandas as pd
import numpy as np

# Preprocessing clinical data

We do this prior to upload because of patient merging issues.

In [ ]:
from redcap_preprocessing import redcap_preprocessing

redcap_filepath = '/Users/gustaveronteix/Documents/data/OraklOnco/clinical/upload/ColonGR-PDO_DATA_2023-10-18_1504.csv'
output_dir = '/Users/gustaveronteix/Documents/data/OraklOnco/clinical/upload/single_files'
conversion_table_filepath = 'conversion_table/redcap_CRC_conversion_table.csv'
disease_type = 'CRC'

redcap_preprocessing.preprocess_redcap_data(redcap_filepath,
                                            conversion_table_filepath,
                                            output_dir,
                                            disease_type,
                                            True,)

In [2]:
from redcap_preprocessing import redcap_preprocessing

redcap_filepath = '/Users/gustaveronteix/Documents/data/OraklOnco/clinical/upload/ColonGR-PDO_DATA_2023-10-18_1504.csv'
output_dir = '/Users/gustaveronteix/Documents/data/OraklOnco/clinical/upload/single_files'
conversion_table_filepath = 'conversion_table/redcap_CRC_conversion_table.csv'
disease_type = 'CRC'

redcap_preprocessing.preprocess_redcap_data(redcap_filepath,
                                            disease_type=disease_type,
                                            output_dir=output_dir,
                                            save_as_single_file = False)

(26, 'GR0069')
(26, 'GR0069')
(38, 'GR0001')
(38, 'GR0001')
(39, 'GR0002 ')
(39, 'GR0002 ')
(39, 'GR0060 ')
(39, 'GR0060 ')
(40, 'GR0004')
(40, 'GR0004')
(41, 'GR0005')
(41, 'GR0005')
(42, 'GR0006')
(42, 'GR0006')
(43, 'GR0007')
(43, 'GR0007')
(44, 'GR0008')
(44, 'GR0008')
(45, 'GR0009')
(45, 'GR0009')
(46, 'GR0010')
(46, 'GR0010')
(47, 'GR0011')
(47, 'GR0011')
(48, 'GR0013')
(48, 'GR0013')
(49, 'GR0014')
(49, 'GR0014')
(50, 'GR0019')
(50, 'GR0019')
(51, 'GR0021')
(51, 'GR0021')
(52, 'GR0023')
(52, 'GR0023')
There are no PDO cell lines for record_id 53.
There are no PDO cell lines for record_id 53.
(53, '')
(53, 'XX0000')
(54, 'GR0025')
(54, 'GR0025')
(55, 'GR0027')
(55, 'GR0027')
(56, 'GR0029')
(56, 'GR0029')
(58, 'GR0073')
(58, 'GR0073')
(88, 'GR0030')
(88, 'GR0030')
(89, 'GR0032')
(89, 'GR0032')
(90, 'GR0034')
(90, 'GR0034')
(91, 'GR0036')
(91, 'GR0036')
(92, 'GR0039')
(92, 'GR0039')
(93, 'GR0040')
(93, 'GR0040')
(93, 'GR0118')
(93, 'GR0118')
(94, 'GR0041')
(94, 'GR0041')
(95, 'GR00

In [4]:
from redcap_preprocessing import redcap_preprocessing

redcap_filepath = '/Users/gustaveronteix/Documents/data/OraklOnco/clinical/upload/PANCREASGR-PDO_DATA_2023-10-18_1509.csv'
output_dir = '/Users/gustaveronteix/Documents/data/OraklOnco/clinical/upload/single_files'
conversion_table_filepath = 'conversion_table/redcap_PGR_conversion_table.csv'
disease_type = 'PDAC'

redcap_preprocessing.preprocess_redcap_data(redcap_filepath,
                                            disease_type=disease_type,
                                            output_dir=output_dir,
                                            save_as_single_file = False)

(9, 'GR75')
(9, 'GR0075')
(9, 'GR96')
(9, 'GR0096')
There are no PDO cell lines for record_id 12.
There are no PDO cell lines for record_id 12.
(12, '')
(12, 'XX0000')
(22, 'GR0005')
(22, 'GR0005')
There are no PDO cell lines for record_id 101.
There are no PDO cell lines for record_id 101.
(101, '')
(101, 'XX0001')
(106, 'GR0000')
(106, 'GR0000')
(338, 'GR0002')
(338, 'GR0002')
There are no PDO cell lines for record_id 450.
There are no PDO cell lines for record_id 450.
(450, '')
(450, 'XX0002')
(507, 'GR59')
(507, 'GR0059')
(507, 'GR65')
(507, 'GR0065')
(591, 'GR0055')
(591, 'GR0055')
(685, 'GR0045')
(685, 'GR0045')
(706, 'GR0001')
(706, 'GR0001')
(707, 'GR0003')
(707, 'GR0003')
(708, 'GR0004')
(708, 'GR0004')
(709, 'GR0008')
(709, 'GR0008')
(710, 'GR0009')
(710, 'GR0009')
(711, 'GR0010')
(711, 'GR0010')
(712, 'GR0011')
(712, 'GR0011')
(713, 'GR0012')
(713, 'GR0012')
(719, 'GR0026')
(719, 'GR0026')
(742, 'GR0023')
(742, 'GR0023')
(744, 'GR0013')
(744, 'GR0013')
(745, 'GR0014')
(745, 